In [ ]:
# Credit goes to the top solution of the previous earthquake competition

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from glob import glob

import os
import dask.dataframe as dd

from tsfresh.feature_extraction import feature_calculators
import librosa
import pywt

from joblib import Parallel, delayed
from tqdm import tqdm_notebook
import scipy as sp
import itertools
import gc

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
import gc

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')
train['segment_id'] = train['segment_id'].astype(str)
train = train.sort_values('segment_id')
y = train['time_to_eruption']
del(train)

In [ ]:
train_paths = sorted(glob('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/*'))

In [ ]:
sub = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')

In [ ]:
#+- 600
# std 100,800
# plt.plot(train_sample['sensor_1'].rolling(10).std())

In [ ]:
np.random.seed(1337)
noise = np.random.normal(-10, 200, 60_001)


def denoise_signal_simple(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    #univeral threshold
    uthresh = 10
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')


def feature_gen(path):
    X = (pd.read_csv(path).fillna(200).values.T + noise).T
    z = X - np.median(X,axis=0)
    features = []
    for i in range(X.shape[1]):
        sig = z[:,i]
        den_sample_simple = denoise_signal_simple(sig)
        mfcc = librosa.feature.mfcc(sig)
        mfcc_mean = mfcc.mean(axis=1)
        percentile_roll50_std_20 = np.percentile(pd.Series(sig).rolling(50).std().dropna().values, 20)
        features.extend([feature_calculators.number_peaks(den_sample_simple, 2),percentile_roll50_std_20,mfcc_mean[18],mfcc_mean[4]])
    return features

In [ ]:
%%time
train_samples = [Parallel(n_jobs=4)(delayed(feature_gen)(train_path) for train_path in tqdm_notebook(train_paths))]

In [ ]:
train_samples = np.array(train_samples).reshape(-1,40)
# del(samples)
del(train_paths)
gc.collect()

In [ ]:
cat = CatBoostRegressor(loss_function='MAE',iterations = 5000)
cat.fit(train_samples,y)

In [ ]:
print(mean_absolute_error(y,cat.predict(train_samples)))

In [ ]:
test_paths = sorted(glob('/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/*'))

In [ ]:
%%time
test_samples = [Parallel(n_jobs=4)(delayed(feature_gen)(test_path) for test_path in tqdm_notebook(test_paths))]

In [ ]:
# len(train_paths), len(test_paths)

In [ ]:
test_samples = np.array(test_samples).reshape(-1,40)
del(test_paths)
gc.collect()

In [ ]:
preds = cat.predict(test_samples)

In [ ]:
sub['time_to_eruption'] = preds

In [ ]:
sub.to_csv('submission.csv', header=True, index=False)